### Instruction

We provide evaluation metrics for reconstruction, generation, and optimizatioin tasks. Please check our paper for more details. The to-be-evaluated dataset should be under "--root_path" with a format like "eval_recon.pt". The folder containing the pre-trained property prediction model checkpoint should be under "./prop_models".

In [1]:
%run compute_metrics.py --root_path my_data --eval_model_name my_model --tasks recon gen opt

/home/qutd/anaconda3/envs/cdvae/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 3785/3785 [00:09<00:00, 403.98it/s]


{'comp_valid': 0.9853, 'struct_valid': 1.0, 'valid': 0.9853, 'comp_div': 11.837321359714958, 'struct_div': 0.595585990748511, 'wdist_density': 0.09399001530644414, 'wdist_num_elems': 0.1649550858652577, 'wdist_prop': 0.029328641319841833}


100%|██████████| 9853/9853 [00:25<00:00, 386.97it/s]


{'match_rate': 0.0, 'rms_dist': nan, 'comp_valid': 0.9853, 'struct_valid': 1.0, 'valid': 0.9853, 'comp_div': 11.837321359714958, 'struct_div': 0.595585990748511, 'wdist_density': 0.09399001530644414, 'wdist_num_elems': 0.1649550858652577, 'wdist_prop': 0.029328641319841833, 'cov_recall': 0.9955085865257596, 'cov_precision': 0.978, 'amsd_recall': 0.04856292904575247, 'amsd_precision': 0.059185676036623044, 'amcd_recall': 0.7534509017900184, 'amcd_precision': 1.287651497669196, 'SR5': 0.96, 'SR10': 1.0, 'SR15': 1.0}


### Evaluation Metrics

- Reconstruction: we evaluate reconstruction performance by matching the generated structure and the input structure for all materials in the test set. We use StructureMatcher from pymatgen \[1\] following CDVAE \[2\], which finds the best match between two structures considering all invariances of materials. The match rate is the percentage of materials satisfying the criteria stol=0.5, angle tol=10, ltol=0.3. The RMSE is averaged over all matched materials.

- Generation: we apply several physically meaningful metrics to evaluate the validity, property statistics, and diversity of generated materials following CDVAE \[2\]. (1) Validity. A structure is valid as long as the shortest distance between any pair of atoms is larger than 0.5 A, which is a relative weak criterion. The composition is valid if the overall charge is neutral as computed by SMACT \[3\]. (2) Coverage (COV). COV-R (Recall) and COV-P (Precision) are used to measure the similarity between ensembles of generated materials and ground truth materials in test set. Intuitively, COV-R measures the percentage of ground truth materials being correctly predicted, and COV-P measures the percentage of predicted materials having high quality. (3) Property statistics. We compute the earth mover’s distance (EMD) between the property distribution of generated materials and test materials. We use density, energy predicted by an independent GNN, and number of unique elements as our properties.

- Property optimization: we jointly train a property predictor parameterized by an MLP to predict properties of training materials from the latents. To optimize properties, we start with the latent representations of testing materials and apply gradient ascent in the latent space to improve the predicted property. After applying 5000 gradient steps, 10 materials are decoded from the latent trajectories every 500 steps. We use an independently trained property predictor to select the best one from the 10 decoded materials. We report the success
rate (SR) as the percentage of materials achieving 5, 10, and 15 percentiles of the target property
distribution.

\[1\] Ong, Shyue Ping, et al. "Python Materials Genomics (pymatgen): A robust, open-source python library for materials analysis." Computational Materials Science 68 (2013): 314-319.

\[2\] Xie, Tian, et al. "Crystal diffusion variational autoencoder for periodic material generation." arXiv preprint arXiv:2110.06197 (2021).

\[3\] Davies, Daniel W., et al. "SMACT: Semiconducting materials by analogy and chemical theory." Journal of Open Source Software 4.38 (2019): 1361.